In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
import matplotlib.lines as mlines
from scipy.interpolate import UnivariateSpline
from tqdm import tqdm

In [3]:
from MCEq.version import __version__

print(__version__)

1.2.6


In [4]:
from MCEq.core import MCEqRun
import mceq_config as config
import crflux.models as pm

In [5]:
#config.debug_level = 2
#config.enable_default_tracking = False
#config.mceq_db_fname = "mceq_db_lext_dpm193_v150_sstar.h5"
#config.adv_set["disable_charm_pprod"] = True

In [6]:
# Average mass number may be wrong but not relevant
mceq_air = MCEqRun(
    interaction_model="SIBYLL23C",
    theta_deg=0.0,
    primary_model=(pm.GlobalSplineFitBeta, None),
    #     density_model = ('MSIS00',('SoudanMine','January')),
)

MCEqRun::set_interaction_model(): SIBYLL23C
ParticleManager::_init_default_tracking(): Initializing default tracking categories (pi, K, mu)
MCEqRun::set_density_model(): Setting density profile to CORSIKA ('BK_USStd', None)
MCEqRun::set_primary_model(): GlobalSplineFitBeta 


In [7]:
mceq_air.set_interaction_model("SIBYLL23C", force=True)
# inject_scenario(pC_scen, mceq_air)
#mceq_air.set_primary_model(
#    pm.GlobalSplineFitBeta(None),
#)  # pm.GaisserStanevTilav('4-gen'))#
mceq_air.set_theta_deg(0)

MCEqRun::set_interaction_model(): SIBYLL23C
MCEqRun::set_primary_model(): GlobalSplineFitBeta 


In [8]:
cos_thetas = np.arange(0.0, 1.01, 0.1)
thetas = np.degrees(np.arccos(cos_thetas))
print(cos_thetas, thetas)

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ] [90.         84.26082952 78.46304097 72.54239688 66.42182152 60.
 53.13010235 45.572996   36.86989765 25.84193276  0.        ]


In [9]:
surface_flux_GSF = []
surface_flux_GST = []
for ia, theta in enumerate(tqdm(thetas)): # loop over angles
    mceq_air.set_theta_deg(theta)
    #mceq_air.set_primary_model(pm.GlobalSplineFitBeta(None))
    mceq_air.solve()
    surface_flux_GSF.append(
        mceq_air.get_solution("mu+", mag=0) + mceq_air.get_solution("mu-", mag=0)
    )
    mceq_air.set_primary_model(pm.GaisserStanevTilav("4-gen"))
    mceq_air.solve()
    surface_flux_GST.append(
        mceq_air.get_solution("mu+", mag=0) + mceq_air.get_solution("mu-", mag=0)
    )

  0%|          | 0/11 [00:32<?, ?it/s]


TypeError: MCEqRun.set_primary_model() missing 1 required positional argument: 'tag'

In [ ]:
config.debug_level = 0

In [ ]:
cr_grid = mceq_air.e_grid[30:-10] # cut because higher energies are not relevant for analysis

In [ ]:
ground_muspec_prim_energies = []
for ei, eprim in enumerate(tqdm(cr_grid)):
    ground_muspec_prim_energies.append(np.zeros((thetas.shape[0], mceq_air.dim)))
    mceq_air.set_single_primary_particle(E=eprim, pdg_id=2212)
    for ia, theta in tqdm(enumerate(thetas)):
        mceq_air.set_theta_deg(theta)
        mceq_air.solve()
        ground_muspec_prim_energies[-1][ia, :] = mceq_air.get_solution(
            "mu+", mag=0
        ) + mceq_air.get_solution("mu-", mag=0)

11it [00:51,  4.65s/it]00:00<?, ?it/s]
11it [00:47,  4.36s/it]00:51<1:08:15, 51.20s/it]
11it [00:52,  4.74s/it]01:39<1:04:54, 49.30s/it]
11it [00:50,  4.63s/it]02:31<1:05:46, 50.59s/it]
11it [00:52,  4.79s/it]03:22<1:05:05, 50.72s/it]
11it [00:53,  4.90s/it]04:14<1:05:09, 51.44s/it]
11it [00:52,  4.80s/it]05:08<1:05:22, 52.29s/it]
11it [00:53,  4.90s/it]06:01<1:04:43, 52.47s/it]
11it [00:53,  4.88s/it]06:55<1:04:24, 52.94s/it]
11it [00:51,  4.67s/it]07:49<1:03:48, 53.17s/it]
11it [00:52,  4.74s/it][08:40<1:02:15, 52.61s/it]
11it [00:52,  4.74s/it][09:32<1:01:13, 52.48s/it]
11it [00:53,  4.82s/it][10:25<1:00:13, 52.37s/it]
11it [00:52,  4.74s/it][11:18<59:34, 52.57s/it]  
11it [00:53,  4.83s/it][12:10<58:33, 52.44s/it]
11it [00:53,  4.91s/it][13:03<57:53, 52.64s/it]
11it [00:51,  4.65s/it][13:57<57:28, 53.05s/it]
11it [00:53,  4.85s/it][14:48<55:57, 52.46s/it]
11it [00:51,  4.65s/it][15:41<55:21, 52.72s/it]
11it [00:52,  4.82s/it][16:32<53:58, 52.24s/it]
11it [00:51,  4.69s/it][17:25<53

In [ ]:
import pickle

pickle.dump(
    [mceq_air.e_grid, cos_thetas, cr_grid, ground_muspec_prim_energies],
    open("ground_muspec_prim_energies.pkl", "wb"),
)
pickle.dump(
    [mceq_air.e_grid, cos_thetas, surface_flux_GSF, surface_flux_GST],
    open("surface_fluxes.pkl", "wb"),
)

In [ ]:
import pickle

# pickle.dump([mceq_air.e_grid, angles, cr_grid, ground_muspec_prim_energies], open('ground_muspec_prim_energies.pkl', 'wb'))
pickle.dump(
    [mceq_air.e_grid, cos_thetas, surface_flux_GSF, surface_flux_GST],
    open("surface_fluxes.pkl", "wb"),
)